In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('train.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 22 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: floa

In [7]:
test = pd.read_csv('test.csv') # please pay attantion that we don't have target on test sample

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 21 columns):
_id               16476 non-null object
age               16476 non-null int64
job               16476 non-null object
marital           16476 non-null object
education         16476 non-null object
default           16476 non-null object
housing           16476 non-null object
loan              16476 non-null object
contact           16476 non-null object
month             16476 non-null object
day_of_week       16476 non-null object
duration          16476 non-null int64
campaign          16476 non-null int64
pdays             16476 non-null int64
previous          16476 non-null int64
poutcome          16476 non-null object
emp.var.rate      16476 non-null float64
cons.price.idx    16476 non-null float64
cons.conf.idx     16476 non-null float64
euribor3m         16476 non-null float64
nr.employed       16476 non-null float64
dtypes: float64(5), int64(5), object(11)
memory usa

In [9]:
train[train['campaign'] == 0] # means all clients have at least one call from bank

,_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target


In [10]:
train['campaign'].describe() # means all clients have at least one call from bank

count    24712.000000
mean         2.580163
std          2.776757
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         43.000000
Name: campaign, dtype: float64

In [13]:
train[train['target'] == 1]['campaign'].describe() # looks like count of calls doesn't matter a lot. 75% with target = 1 has just 2 calls

count    2787.000000
mean        2.049157
std         1.617637
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max        17.000000
Name: campaign, dtype: float64

In [12]:
train['duration'].describe()

count    24712.000000
mean       258.345298
std        261.573344
min          0.000000
25%        102.000000
50%        180.000000
75%        319.000000
max       4918.000000
Name: duration, dtype: float64

In [14]:
train[train['target'] == 1]['duration'].describe() # but duration means a lot as described in overview

count    2787.000000
mean      551.896304
std       404.420029
min        37.000000
25%       255.000000
50%       442.000000
75%       739.500000
max      4199.000000
Name: duration, dtype: float64